Ignis: Text Pre-processing
==========================

In [1]:
import collections
import glob
import pathlib
import re
import threading
import time

import gensim
import tqdm

In [2]:
# Jupyter notebook setup
import ipywidgets as widgets
from IPython.core.display import display, HTML

# Custom styling:
# - Prevent vertical scrollbars in output subareas
style = """
<style>
   .jupyter-widgets-output-area .output_scroll {
        height: unset !important;
        border-radius: unset !important;
        -webkit-box-shadow: unset !important;
        box-shadow: unset !important;
    }
    .jupyter-widgets-output-area  {
        height: auto !important;
    }
</style>
"""
display(HTML(style))

Data ingestion
--------------------

We will track the contents and filename of each document, then tokenise them all and feed them into an `ignis.Corpus` that will be saved.

We should, by all accounts, actually be preparing a separate text cleaning function and running the raw text through it immediately, but this way we can see the effects of each step of the data cleaning.

In [3]:
raw_files = glob.glob("./data/bbc/*/*.txt")

In [4]:
RawDocument = collections.namedtuple("RawDocument", "metadata, tokens, human_readable")

In [5]:
raw_docs = []
for file in tqdm.tqdm(raw_files):
    filename = pathlib.Path(file).as_posix()

    metadata = {"filename": filename}

    with open(file) as f:
        tokens = f.read()

    raw_doc = RawDocument(metadata, tokens, human_readable=tokens)

    raw_docs.append(raw_doc)

100%|██████████| 2225/2225 [00:00<00:00, 8656.44it/s]


In [6]:
def show_raw_doc(doc_id=0):
    print(raw_docs[doc_id].metadata)
    print()
    print(raw_docs[doc_id].tokens)


widgets.interact(show_raw_doc, doc_id=(0, len(raw_docs) - 1))

interactive(children=(IntSlider(value=0, description='doc_id', max=2224), Output()), _dom_classes=('widget-int…

<function __main__.show_raw_doc(doc_id=0)>

Text pre-processing and tokenisation
------

### Naive tokenisation (by whitespace)
- Case folding
- Strip leading/trailing non-informative punctuation from tokens
- Remove single apostrophes
- Remove single brackets within words
  - For dealing with cases like "the recipient(s)" -- Which will get tokenised to "the recipient(s" otherwise

In [7]:
strip_punctuation = "'\"()[]<>?!,.:;/|_"
bracket_pairs = [
    ["(", ")"],
    ["[", "]"],
]


def naive_tokenise(doc):
    """
    Naively tokenises a document.
    
    Returns
    -------
    str
        The document as a string of space-separated tokens
    """
    new_tokens = []

    tokens = doc.split()
    for token in tokens:
        token = token.casefold()
        token = token.strip(strip_punctuation)
        token = token.replace("'", "")

        for bracket_pair in bracket_pairs:
            if bracket_pair[0] in token and bracket_pair[1] not in token:
                token = token.replace(bracket_pair[0], "")
            if bracket_pair[1] in token and bracket_pair[0] not in token:
                token = token.replace(bracket_pair[1], "")

        if token != "":
            new_tokens.append(token)

    return new_tokens

In [8]:
naive_tokenise('This is a t(e)st of the system\'s "tokenisation" operation(s).')

['this',
 'is',
 'a',
 't(e)st',
 'of',
 'the',
 'systems',
 'tokenisation',
 'operations']

In [9]:
naive_docs = []
for raw_doc in raw_docs:
    naive_docs.append(raw_doc._replace(tokens=naive_tokenise(raw_doc.tokens)))

In [10]:
def show_naive_doc(doc_id=0):
    print(naive_docs[doc_id].metadata)
    print()
    print(" ".join(naive_docs[doc_id].tokens))


widgets.interact(show_naive_doc, doc_id=(0, len(naive_docs) - 1))

interactive(children=(IntSlider(value=0, description='doc_id', max=2224), Output()), _dom_classes=('widget-int…

<function __main__.show_naive_doc(doc_id=0)>

### Automated n-gram detection

Chunk into significant bigrams based on collocation frequency

(N.B.: Gensim implies that the input to the Phraser should be a list of single sentences, but we will feed it a list of documents instead.)

- Min count: How many documents the n-grams need to appear in

- Scoring: "default" or "npmi"

- Threshold: Intuitively, higher threshold means fewer phrases.
  - With the default scorer, this is greater than or equal to 0; with the NPMI scorer, this is in the range -1 to 1.

- Common terms: These terms will be ignored if they come between normal words.
  - E.g., if `common_terms` includes the word "of", then when the phraser sees "Wheel of Fortune" it actually evaluates _"Wheel Fortune"_ as an n-gram, putting "of" back in only at the output level to give _wheel_of_fortune_.
  - With the `common_terms` option set properly,  there do not seem to be many significant trigrams and above left to pick up -- We stick with bigrams to save on runtime.

In [11]:
min_count = 5
scoring = "npmi"
# We want a relatively high threshold so that we don't start littering spurious n-grams all over our corpus, diluting our results.
# E.g., we want "Lord_of_the_Rings", but not "slightly_better_than_analysts"
threshold = 0.7
common_terms = ["a", "an", "the", "of", "on", "in", "at"]

This could take a while, so set up a threaded function with a basic progress indicator in the main thread

In [12]:
def find_phrases(docs, results, trigrams=False):
    # Build, finalise, and apply the bigram model
    bigram_model = gensim.models.Phrases(
        docs,
        min_count=min_count,
        threshold=threshold,
        scoring=scoring,
        common_terms=common_terms,
    )
    bigram_model = gensim.models.phrases.Phraser(bigram_model)

    bigram_docs = bigram_model[docs]

    results[0] = bigram_docs

    if trigrams:
        # Repeat to get trigrams
        trigram_model = gensim.models.Phrases(
            bigram_docs,
            min_count=min_count,
            threshold=threshold,
            scoring=scoring,
            common_terms=common_terms,
        )
        trigram_model = gensim.models.phrases.Phraser(trigram_model)

        trigram_docs = trigram_model[docs]

        results[0] = trigram_docs

In [13]:
print("Finding phrases", flush=True, end="")
start_time = time.perf_counter()

# Just send the textual content through the Phraser, not the document metadata
for_phrasing = [naive_doc.tokens for naive_doc in naive_docs]

# Will contain the documents after trigram processing
results = [None]
t = threading.Thread(target=find_phrases, args=(for_phrasing, results, False))
t.start()

progress_countdown = 1.0

while t.isAlive():
    time.sleep(0.1)
    progress_countdown -= 0.1
    if progress_countdown <= 0:
        print(" .", flush=True, end="")
        progress_countdown = 1

elapsed = time.perf_counter() - start_time
print(f" Done. ({elapsed:.3f}s)")

after_phrasing = results[0]

# Put metadata back in
phrased_docs = []
for index, tokens in enumerate(after_phrasing):
    phrased_docs.append(naive_docs[index]._replace(tokens=tokens))

Finding phrases . . . . Done. (4.852s)


In [14]:
def show_phrased_doc(doc_id=0):
    print(phrased_docs[doc_id].metadata)
    print()
    print(" ".join(phrased_docs[doc_id].tokens))


widgets.interact(show_phrased_doc, doc_id=(0, len(phrased_docs) - 1))

interactive(children=(IntSlider(value=0, description='doc_id', max=2224), Output()), _dom_classes=('widget-int…

<function __main__.show_phrased_doc(doc_id=0)>

In [15]:
# Trigrams and above in the corpus (with `common_terms` set properly)
seen_tokens = set()
for document in phrased_docs:
    for token in document.tokens:
        if token.count("_") >= 2:
            if token not in seen_tokens:
                print(token)
                seen_tokens.add(token)

lord_of_the_rings
house_of_lords
tens_of_thousands
chambers_of_commerce
billions_of_dollars
weapons_of_mass
tip_of_the_iceberg
prisoner_of_azkaban
tells_the_story
shaun_of_the_dead
pirates_of_the_caribbean
passion_of_the_christ
meet_the_fockers
dismantle_an_atomic
hall_of_fame
phantom_of_the_opera
hepburn_in_the_aviator
miscarriages_of_justice
convention_on_human
hands_of_extremists
code_of_conduct
archbishop_of_canterbury
prevention_of_terrorism
tests_in_tel
faking_a_motorcycle
icing_on_the_cake
capt_a_persico
blogs_in_existence
hitting_the_shelves


### Post-phrasing cleaning

- Remove stop words (optional)
- Remove purely numeric/non-alphabetic/single-character tokens
  - Under the assumption that significant tokens, like the "19" in "Covid 19" or the "11" in "Chapter 11 (bankruptcy)" would have been picked up by the phraser

In [16]:
# Not needed if using term weighting, but we could use stopsets from NLTK or other sources
stopset = []


def second_tokenise(tokens):
    new_tokens = []
    for token in tokens:
        if token in stopset or re.match("^[^a-z]+$", token) or len(token) <= 1:
            continue
        new_tokens.append(token)

    return new_tokens

In [17]:
final_docs = []
for phrased_doc in phrased_docs:
    final_docs.append(phrased_doc._replace(tokens=second_tokenise(phrased_doc.tokens)))

In [18]:
def show_final_doc(doc_id=0):
    print(final_docs[doc_id].metadata)
    print()
    print(" ".join(final_docs[doc_id].tokens))


widgets.interact(show_final_doc, doc_id=(0, len(final_docs) - 1))

interactive(children=(IntSlider(value=0, description='doc_id', max=2224), Output()), _dom_classes=('widget-int…

<function __main__.show_final_doc(doc_id=0)>

In [19]:
# Simple deduplication
seen_docs = set()
deduped_docs = []
for doc in final_docs:
    # Cast the document tokens as a tuple so that we can use it as a deduplicating hash
    doc_hash = tuple(doc.tokens)
    if doc_hash in seen_docs:
        # print(f"Duplicate document: {doc[0]['filename']}")
        pass
    else:
        seen_docs.add(doc_hash)
        deduped_docs.append(doc)

Save to Ignis Corpus
----

In [20]:
import ignis

In [21]:
corpus = ignis.Corpus()

for doc in deduped_docs:
    corpus.add_doc(**doc._asdict())
corpus.save("bbc-full.corpus")

In [22]:
# And make sure it loads without errors as well.
corpus = ignis.load_corpus("bbc-full.corpus")

In [23]:
corpus_doc_ids = list(corpus.documents.keys())


def show_corpus_doc(index=0):
    doc = corpus.documents[corpus_doc_ids[index]]
    print(doc.metadata)
    print()
    print("-" * 10)
    print()
    print(doc.human_readable)
    print()
    print("-" * 10)
    print()
    print(" ".join(doc.tokens))


widgets.interact(show_corpus_doc, index=(0, len(corpus_doc_ids) - 1))

interactive(children=(IntSlider(value=0, description='index', max=2121), Output()), _dom_classes=('widget-inte…

<function __main__.show_corpus_doc(index=0)>